In [3]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.0


In [4]:
!python -V

Python 3.10.16


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/mustafacavusoglu/miniconda3/envs/mlops/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/mustafacavusoglu/miniconda3/envs/mlops/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
y_pred.std()

np.float64(6.247488852238703)

In [17]:
year = 2023
month = 3
output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'

In [19]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [24]:
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'prediction_duration': y_pred
})

In [25]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [26]:
!ls -lh output

total 163848
-rw-r--r--@ 1 mustafacavusoglu  staff    65M Jun 12 15:57 yellow_tripdata_2023-03.parquet
